In [2]:
import pandas as pd
import numpy as np
import ast 
from datetime import datetime

In [3]:
path_movies=(r"C:\Users\gaston\Documents\estudio\material henry compartido\Proyecto integrador\movies_dataset.csv")
df_movies=pd.read_csv(path_movies, encoding="utf-8")
df_movies["belongs_to_collection"].head()


C:\Users\gaston\AppData\Local\Temp\ipykernel_2788\348715108.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies=pd.read_csv(path_movies, encoding="utf-8")


0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [4]:
# path del archivo
path_credits=r"C:\Users\gaston\Documents\estudio\material henry compartido\Proyecto integrador\credits.csv"
# Leer el CSV con pandas
df_credits=pd.read_csv(path_credits,encoding="utf-8")



### FUNCIONES PARA LA DESANIDAR LOS DATOS(NAMES) QUE SE EN ENCUENTRAN EN DATASET DE MOVIES

In [5]:
#funcion para desanidar los datos que se encuentran en forma de diccionario

def explode_dict(value,key): # Asignamos las variables: 'value' seria el valor de la columna y 'key' la clave que queremos extrer
    if pd.isna(value):
        return np.nan
    try:
        dicc = ast.literal_eval(value)  # Convertimos el valor string en un diccionario
        if isinstance(dicc, dict):
            return dicc.get(key, np.nan)  # Extraemos únicamente el valor asociado a la clave 'key'
        else:
            return np.nan
    except (ValueError, SyntaxError):
        return np.nan

In [6]:
#  funcion para datos anidados con listas de diccionarios 
def explode_list(value, key):
    if pd.isna(value):
        return np.nan
    try:
        lis = ast.literal_eval(value)
        if isinstance(lis, list): 
            names = [item.get(key) for item in lis if isinstance(item, dict)]
            return ', '.join(names) if names else np.nan  # Esto devuelve una cadena con los valores separados por comas
        else:
            return np.nan
    except (ValueError, SyntaxError):
        return np.nan

### FUNCIONES PARA LA DESANIDAR LOS DATOS QUE SERAN UTILIZADOS EN EL ENDPOINT_1 Y ENDPOINT_2

In [7]:
#funcion para extraer nombres de actores
def extract_actors(columna):
    try:
        cast = ast.literal_eval(columna)
    except (ValueError, SyntaxError):
        return []
    
    return [actor.get('name') for actor in cast]

# Y aplicamos


In [8]:
#función para extraer los directores 
def extract_directors(crew_list):
    if isinstance(crew_list, str):
        crew_list = ast.literal_eval(crew_list)
    
    directores = [d['name'] for d in crew_list if d.get('job') == 'Director']
    return directores[0] if directores else None

In [9]:
"""def explode_list_2(value, keys):
    try:
        lis = ast.literal_eval(value) if isinstance(value, str) else value  # Convierte cadenas en listas/diccionarios
        if not isinstance(lis, list):  
            return np.nan  # Si no es una lista, retorna NaN
        
        extracted_values = []
        for item in lis:
            if isinstance(item, dict):
                extracted_values.append(tuple(item.get(int(key), item.get(str(key), np.nan)) for key in keys))
        
        return extracted_values if extracted_values else np.nan
    except (ValueError, SyntaxError, TypeError):  
        return np.nan """

'def explode_list_2(value, keys):\n    try:\n        lis = ast.literal_eval(value) if isinstance(value, str) else value  # Convierte cadenas en listas/diccionarios\n        if not isinstance(lis, list):  \n            return np.nan  # Si no es una lista, retorna NaN\n        \n        extracted_values = []\n        for item in lis:\n            if isinstance(item, dict):\n                extracted_values.append(tuple(item.get(int(key), item.get(str(key), np.nan)) for key in keys))\n        \n        return extracted_values if extracted_values else np.nan\n    except (ValueError, SyntaxError, TypeError):  \n        return np.nan '

In [10]:
pd.set_option("display.max_columns", None)  # Muestra todas las columnas
pd.set_option("display.max_colwidth", None)  # No trunca el contenido de cada celda
data=df_credits["crew"]
data

0        [{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67'

In [11]:
print(df_credits["crew"].isna().sum())
print(df_credits["cast"].isna().sum())

df_credits = df_credits.dropna(subset=["crew", "cast", "id"], how="any")
for fila in df_credits.index:  
    if df_credits.loc[fila, ["crew", "cast", "id"]].isnull().any():  
        df_credits.drop(index=fila, inplace=True)

print(df_credits["crew"].isna().sum())
print(df_credits["cast"].isna().sum())



0
0
0
0


In [12]:
# Aplicamos la funcion para extraer los 'name' o nombres que hacen referencia al dato mas importante y luego se elimina la columna
keys_cast=["cast_id","character","credit_id","gender","id","name","order","profile_path"]
keys_crew=["credit_id","department","gender","id","job","name","profile_path","department","job","name","profile_path"]

key="name"
#
#df_crew_data=df_credits["crew"].apply(explode_list(), keys=keys_crew)#obtenemos el nombre del actor listas de diccionarios del crew)
#df_cast_data=df_credits["cast"].apply(explode_list(), keys=keys_cast)#obtenemos las listas de diccionarios del cast

#df_crew_data = df_crew_data.explode().apply(pd.Series)#expande la serie 
#df_crew_data.columns=keys_crew# se asignan los indices de los dic como columnas 

#df_cast_data = df_cast_data.explode().apply(pd.Series)#expande la serie 
#df_cast_data.columns=keys_cast# se asignan los indices de los dic como columnas 


df_movies["belong_to_collection"]=df_movies["belongs_to_collection"].apply(explode_dict,key="name")

df_movies["genres"] = df_movies["genres"].apply(explode_list, key=key)

df_movies["countries"] = df_movies["production_countries"].apply(explode_list, key=key)

df_movies["companies"] = df_movies["production_companies"].apply(explode_list, key=key)

df_movies["languages"] = df_movies["spoken_languages"].apply(explode_list, key=key)

df_credits["actors"]=df_credits["cast"].apply(extract_actors)

df_credits["director"]=df_credits["crew"].apply(extract_directors)






# eleiminamos la informacion que no utilizaremos, tanto en credits como en movies

df_movies=df_movies.drop(columns=["production_companies","production_countries","spoken_languages"]) #columnas que ya han sido desanidadas 

df_credits=df_credits.drop(columns=["crew","cast"]) #columnas que ya han sido desanidadas(name:actors and director)

#Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

df_movies = df_movies.drop(columns=["video", "imdb_id","adult","original_title","poster_path","homepage"])




In [13]:
df_movies["genres"]

0         Animation, Comedy, Family
1        Adventure, Fantasy, Family
2                   Romance, Comedy
3            Comedy, Drama, Romance
4                            Comedy
                    ...            
45461                 Drama, Family
45462                         Drama
45463       Action, Drama, Thriller
45464                           NaN
45465                           NaN
Name: genres, Length: 45466, dtype: object

In [14]:
#Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
print(df_movies["revenue"].isnull().sum())
print(df_movies["budget"].isnull().sum())
if(df_movies["revenue"].isnull().sum()>=1 or df_movies["revenue"].isna().sum()>=1):
    df_movies["revenue"]=df_movies["revenue"].fillna(0)
    
if(df_movies["budget"].isnull().sum()>=1 or df_movies["budget"].isna().sum()>=1):
    df_movies["budget"]=df_movies["budget"].fillna(0)
print(df_movies["revenue"].isnull().sum())
print(df_movies["budget"].isnull().sum())

6
0
0
0


In [15]:
#Los valores nulos del campo release date deben eliminarse.
print(df_movies["release_date"].isnull().sum())
if(df_movies["release_date"].isnull().sum()>=1):
   
   df_movies= df_movies.dropna(subset=['release_date'])

print("\nDespués de eliminar valores nulos:")
print(df_movies["release_date"].isnull().sum())

87

Después de eliminar valores nulos:
0


In [16]:
def formatoFecha(df):
    valor=False
    dfN=pd.DataFrame()
    dfN_mes=pd.DataFrame()
    dfN[["DD", "MM","AAAA"]] = df.str.split("-""/", expand=True)#Para ver el formato DD-MM o MM-DD

    dfN_mes=dfN[["MM"]].dropna() #eliminar valores nulos

    dfN_mes= dfN_mes.astype(int)#Convertir a int
   #Filtrar donde DD > 12
        
    if ([dfN_mes < 12]):
        valor=True
    return(valor)

In [17]:
# se verifica que columnas contiene fecheas
# se verifica formato de fecha 
regex_fecha1=r"^\d{4}-\d{2}-\d{2}$"
regex_fecha2=r"^\d{2}-\d{2}-\d{4}$"


for columna in df_movies.columns: # Recorrer todas las columnas
    if df_movies[columna].astype(str).str.match(regex_fecha1).any():
        print(f"La columna '{columna}' contiene fechas de formato AAAA-MM-DD.")#formato correcto
        print(df_movies[columna].astype(str).str.match(regex_fecha1).sum())
        formato=1
    elif(df_movies[columna].astype(str).str.match(regex_fecha2).any()):
        print(f"La columna '{columna}' contiene fechas de formato DD-MM-AAAA o MM-DD-AAA.") #puede tener dos formatos
        print(df_movies[columna].astype(str).str.match(regex_fecha2).sum())
        formato=2

#la columna que contiene fecha es la release_date

if(formato==2)and(True==formatoFecha(df_movies["release_date"])):#DD-MM-AAAA
    print("DD-MM-AAAA")

La columna 'id' contiene fechas de formato AAAA-MM-DD.
3
La columna 'release_date' contiene fechas de formato AAAA-MM-DD.
45376


In [18]:
regex_fecha = r"^\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])$"

#  Reemplazar valores nulos o incorrectos con "00-00-0000"
df_año= df_movies["release_date"].apply(lambda x: x if pd.notna(x) and pd.Series(x).str.match(regex_fecha).any() else "00-00-0000")

# Mostrar resultado
df_año= df_año.str.slice(0, 4)#obtengo el año formato string 
df_movies["release_year"] =df_año
df_movies["release_year"]



0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45460    1991
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_year, Length: 45379, dtype: object

In [19]:
print(df_movies["revenue"])

0        373554033.0
1        262797249.0
2                0.0
3         81452156.0
4         76578911.0
            ...     
45460            0.0
45462            0.0
45463            0.0
45464            0.0
45465            0.0
Name: revenue, Length: 45379, dtype: float64


In [20]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
# cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

df_movies["revenue"] = df_movies["revenue"].apply(pd.to_numeric, errors='coerce').fillna(0)
df_movies["budget"] = df_movies["budget"].apply(pd.to_numeric, errors='coerce').fillna(0)

df_movies["return"]= df_movies.apply(
    lambda row: row["revenue"] / row["budget"] if row["budget"]>0 and row["revenue"]>0 else 0,axis=1
)

df_movies["return"]



0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: return, Length: 45379, dtype: float64

In [21]:
# Para analizar posteriormente los datos convertimos los formatos de estas columnas   
df_movies["popularity"] = df_movies["popularity"].apply(pd.to_numeric, errors='coerce').fillna(0) #pasamos a float los valores 
df_movies["release_date"]=df_movies["release_date"].apply(pd.to_datetime,errors='coerce')#pasamos a formato datetime las fechas que se encuentrar en formato de cadena

In [22]:
df_movies["budget"]

0        30000000.0
1        65000000.0
2               0.0
3        16000000.0
4               0.0
            ...    
45460           0.0
45462           0.0
45463           0.0
45464           0.0
45465           0.0
Name: budget, Length: 45379, dtype: float64

In [23]:
print(df_movies.shape)
print(df_credits.shape)


(45379, 21)
(45476, 3)


In [24]:
df = pd.concat([df_movies, df_credits], axis=1) #concatenamos las dos tablas segun su id
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'release_date', 'revenue', 'runtime',
       'status', 'tagline', 'title', 'vote_average', 'vote_count',
       'belong_to_collection', 'countries', 'companies', 'languages',
       'release_year', 'return', 'id', 'actors', 'director'],
      dtype='object')

# ENDPOINT_1

In [25]:
# Generamos una nueva columna con el mes de lanzamiento
df_movies['release_month'] = (df_movies['release_date']).dt.month
# Hacemos un nuevo dataset solamente con las columnas que necesitaremos para el endpoint 1
df_endpoint1 = df_movies[['title','release_month']]
# Remplazamos los valores del mes en numeros por el nombre del mes para luego resulte mas sencilla la funcion 
df_endpoint1['release_month'] = df_endpoint1['release_month'].replace({1:'enero',2:'febrero', 3:'marzo', 4:'abril', 5:'mayo', 6:'junio', 7:'julio', 8:'agosto', 9:'septiembre', 10:'octubre', 11:'noviembre', 12:'diciembre'})

C:\Users\gaston\AppData\Local\Temp\ipykernel_2788\1928382984.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_endpoint1['release_month'] = df_endpoint1['release_month'].replace({1:'enero',2:'febrero', 3:'marzo', 4:'abril', 5:'mayo', 6:'junio', 7:'julio', 8:'agosto', 9:'septiembre', 10:'octubre', 11:'noviembre', 12:'diciembre'})


In [26]:
# Hacemos la funcion que buscara todas las filmaciones que fueron lanzadas en el mes introducido 
def cantidad_filmaciones_mes1(mes:str):
    filmaciones = df_endpoint1['title'][df_endpoint1['release_month'] == mes]
    cantidad = filmaciones.count()
    return f'{cantidad} películas fueron estrenadas en el mes de {mes}'
# Y finalmente probamos la funcion
cantidad_filmaciones_mes1('enero')

'5912 películas fueron estrenadas en el mes de enero'

# ENDPOINT_2

In [27]:
# Generamos una nueva columna pero esta vez colocamos el dia en lugar del mes
df_movies['release_day'] = df_movies['release_date'].dt.day_name()
# Remplazamos los dias de la semana que estan en ingles por su equivalente en español
df_movies['release_day'] = df_movies['release_day'].replace({
    'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles',
    'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado', 'Sunday': 'domingo'
})
# Hacemos un dataframe orientado al uso del endpoint 2
df_endpoint2 = df_movies[['title', 'release_day']]
# Creamos la funcion que consiste en devolver la cantidad de peliculas estrenadas en el dia ingresado
def cantidad_filmaciones_dia(dia:str):
    filmaciones = df_endpoint2['title'][df_endpoint2['release_day'] == dia]
    cantidad = filmaciones.count()
    return f'{cantidad} películas fueron estrenadas en el dia {dia}'
# Probamos dicha funcion
cantidad_filmaciones_dia('lunes')

'3503 películas fueron estrenadas en el dia lunes'

# ENDPOINT_3

In [28]:
# Generamos el dataframe para usar en el endpoint 3
df_endpoint3 = df_movies[['title','popularity','release_year']]
# Generamos la funcion que busca la pelicula en el dataframe y recopila los datos solicitados
def score_titulo(titulo_de_la_filmacion:str):
    film = df_endpoint3[df_endpoint3['title'] == titulo_de_la_filmacion]
    title = film['title'].values[0]
    year = film['release_year'].values[0]
    score = film['popularity'].values[0]
    return f'La película {title} fue estrenada en el año {int(year)} con un score/popularidad de {score}'
# Probamos la funcion
score_titulo('Jumanji')

'La película Jumanji fue estrenada en el año 1995 con un score/popularidad de 17.015539'

# ENDPOINT_4

In [29]:
# Armamos el dataframe para este endpoint
df_endpoint4 = df_movies[['title','release_year','vote_average','vote_count']]
# Hacemos una funcion parecida al endpoint 3 y recopila informacion sobre los votos y otras caracteristicas de la pelicula
def votos_titulo(titulo_de_la_filmacion:str):
    film = df_endpoint4[df_endpoint4['title'] == titulo_de_la_filmacion]
    valoraciones = film['vote_count'].values[0]
    promedio = film['vote_average'].values[0]
    año = film['release_year'].values[0]
    titulo = film['title'].values[0]
    if valoraciones < 2000:
        return 'no cumple con la cantidad de valoraciones minimas'
    else:
        return f'La película {titulo} fue estrenada en el año {int(año)}. La misma cuenta con un total de {valoraciones} valoraciones, con un promedio de {promedio}'
# Nos fijamos si funciona correctamente
votos_titulo('Jumanji')

'La película Jumanji fue estrenada en el año 1995. La misma cuenta con un total de 2413.0 valoraciones, con un promedio de 6.9'

# ENDPOINT 5

In [30]:
# Creamos el dataframe para su uso en el endpoint 5
df_endpoint5 = df[['title', 'actors', 'return']]
# Generamos la funcion donde se filtra las peliculas que contienen el nombre del actor en la lista de actores del df
def get_actor(nombre:str):
    peliculas = df_endpoint5[df_endpoint5['actors'].apply(lambda actors: nombre in actors)] # Aca el filtro de peliculas con el actor mencionado
    pelis = peliculas['title'].tolist()
    retorno = peliculas['return'].tolist()
    cantidad = len(pelis)
    ganancia = sum(retorno)
    promedio = ganancia/cantidad
    return print(f'El actor {nombre} ha participado en {cantidad} filmaciones, el mismo ha conseguido un retorno de {round(ganancia,3)} millones con un promedio de {round(promedio,3)} millones por filmación')
    
# Probamos la funcion creada
get_actor('Adam Sandler')

El actor Adam Sandler ha participado en 47 filmaciones, el mismo ha conseguido un retorno de 72.031 millones con un promedio de 1.533 millones por filmación


# ENDPOINT_6

In [31]:
# Ultimo dataframe para su uso en el endpoint 6
df_endpoint6 = df[['title', 'return','director','release_date','budget','revenue']]
# Esta funcion es mas sencilla que get_actor debido a que en la columna directores solo hay un solo nombre
def get_director(nombre:str):
    peliculas_director = df_endpoint6[df_endpoint6['director'] == nombre]
    if peliculas_director.empty:
        return f"No se encontraron películas para el director: {nombre}"
    retorno_total = peliculas_director['return'].sum()
    informacion_peliculas = peliculas_director[['title', 'release_date', 'return', 'budget', 'revenue']]
    return {
        'nombre_director': nombre,
        'retorno_total': retorno_total,
        'informacion_peliculas': informacion_peliculas
    }
# Finalmente probamos la funcion
get_director('Christopher Nolan')

{'nombre_director': 'Christopher Nolan',
 'retorno_total': np.float64(35.13215293098237),
 'informacion_peliculas':                                         title release_date    return  \
 2466                                Following   1998-09-12  8.080333   
 4099                                  Memento   2000-10-11  4.413677   
 5254                                 Insomnia   2002-05-24  2.472062   
 10122                           Batman Begins   2005-06-10  2.494791   
 11354                            The Prestige   2006-10-19  2.741908   
 12481                         The Dark Knight   2008-07-16  5.430046   
 15480                               Inception   2010-07-14  5.159580   
 18252                   The Dark Knight Rises   2012-07-16  4.339756   
 22878  The Inevitable Defeat of Mister & Pete   2013-10-11  0.000000   
 25896                              Stuntwoman   1977-10-05  0.000000   
 25974            To Grandmother's House We Go   1992-12-06  0.000000   
 44688   

In [32]:
df.to_csv('df_EDA')

In [33]:
#trabajamos solo con las primeras 4000 filas del csv 
df = pd.read_csv(r"C:\Users\gaston\Documents\estudio\material henry compartido\Proyecto integrador\df_EDA")



In [34]:
#librerias
import pandas as pd
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from rake_nltk import Rake

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaston/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
import nltk
nltk.download('punkt')
import nltk
nltk.data.path.append('C:/Users/gaston/nltk_data')
nltk.download('punkt', download_dir='C:/Users/gaston/nltk_data')

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to C:\Users\gaston/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to C:/Users/gaston/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\gaston/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [36]:
# Armamos el dataframe con las columnas necesarias para hacer un sistema de recomendacion con NearestNeighbors
df_reco = df[['title','overview','genres']]

# Reemplazamos valores nulos en 'title', 'overview' y 'generos' con una cadena vacía para que no haya conflictos a la hora de procesar y extraer info
df_reco['title'] = df_reco['title'].fillna('')
df_reco['overview'] = df_reco['overview'].fillna('')
df_reco['genres'] = df_reco['genres'].fillna('')

# Inicializamos el extractor RAKE para insertarla en la siguiente funcion
rake = Rake()

# Creamos la funcion para extraer palabras clave que vamos a aplicar a una de las columnas
def extract_keywords(text):
    rake.extract_keywords_from_text(text)
    return ' '.join(rake.get_ranked_phrases())

# Aplicamos la funcion de extracción de palabras clave a la columna 'overview'
df_reco['keywords'] = df_reco['overview'].apply(extract_keywords)

# Creamos una nueva columna combinando 'title', 'generos' y 'keywords' para tener toda la info necesaria para una buen sistema de recomendacion
df_reco['combined_text'] = df_reco['genres'] + ' ' + df_reco['keywords']

# Aplicamos una vectorizacion con TfidfVectorizer sobre el texto combinado anteriormente
vect = TfidfVectorizer(stop_words='english', max_features=5000)  # Podemos ajustar 'max_features' para limitar la cantidad de palabras
vect_matrix = vect.fit_transform(df_reco['combined_text'])

# Usamos NearestNeighbors con los parametros elegidos y entrenamos el modelo
nn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
nn_model.fit(vect_matrix)

# Hacemos la función de recomendación
def recomendacion(titulo):
    if titulo not in df_reco['title'].values:
        return f"No se encontró la película: {titulo}"

    idx = df_reco[df_reco['title'] == titulo].index[0] # Aca se obtiene el indice de la pelicula ingresada

    distances, indices = nn_model.kneighbors(vect_matrix[idx], n_neighbors=6) # El modelo busca y devuelve los indices de las peliculas similares

    similar_titles = df_reco['title'].iloc[indices[0][1:]]  # Excluimos nuestra pelicula y devolvemos hasta 5 resultados
    
    return similar_titles.tolist()

C:\Users\gaston\AppData\Local\Temp\ipykernel_2788\251848223.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reco['title'] = df_reco['title'].fillna('')
C:\Users\gaston\AppData\Local\Temp\ipykernel_2788\251848223.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reco['overview'] = df_reco['overview'].fillna('')
C:\Users\gaston\AppData\Local\Temp\ipykernel_2788\251848223.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [37]:
# Guardamos los df de cada endpoint y del sistema de recomendacion en formato parquet
df_endpoint1.to_parquet('end1.parquet')
df_endpoint2.to_parquet('end2.parquet')
df_endpoint3.to_parquet('end3.parquet')
df_endpoint4.to_parquet('end4.parquet')
df_endpoint5.to_parquet('end5.parquet')
df_endpoint6.to_parquet('end6.parquet')
df_reco.to_parquet('reco.parquet')